<a href="https://colab.research.google.com/github/skyrag/arn_lab1/blob/main/arn_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import sklearn as sk

mice_eeg = pd.read_csv('EEG_mouse_data_1.csv')
mice_eeg = mice_eeg.iloc[:, 1:26:]

# normalize mice_eeg
mice_eeg = (mice_eeg - mice_eeg.mean()) / mice_eeg.std()
# mice_eeg
mice_eeg_normalize = sk.preprocessing.normalize(mice_eeg)



# shuffle mice_eeg_normalize
mice_eeg_normalize = sk.utils.shuffle(mice_eeg_normalize)
mice_eeg_normalize

array([[-0.07483987, -0.02137731,  0.16643007, ...,  0.11788567,
         0.01514345,  0.25247577],
       [-0.08527989,  0.07871992,  0.10008358, ..., -0.26400653,
        -0.15117786,  0.48908381],
       [-0.08845377, -0.11404897, -0.05399343, ..., -0.02824953,
         0.26264308, -0.16021704],
       ...,
       [ 0.10055929, -0.15899298, -0.10681048, ...,  0.11393393,
         0.28226914,  0.04952303],
       [ 0.06549308, -0.20792627, -0.13599831, ..., -0.18667627,
        -0.24619127,  0.21932519],
       [-0.04154204, -0.01565812, -0.00650638, ..., -0.01492028,
        -0.06947269,  0.07794472]])